# Extraindo informações do site Arbety para pegar o número da sorte 

In [7]:
import time
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Edge
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from bs4 import BeautifulSoup

# Configura as opções do Edge
options = Options()
options.add_argument("--headless")  # Executa o Edge em modo headless

# Configura o serviço do EdgeDriver
service = Service(EdgeChromiumDriverManager().install())

# Cria uma nova instância do Microsoft Edge
driver = Edge(service=service, options=options)

driver.implicitly_wait(10)  # espera até 10 segundos antes de lançar uma NoSuchElementException

url = "https://www.arbety.com/games/double"
driver.get(url)

# Armazena o código HTML atual da página
html_antigo = driver.page_source

try:
    while True:
        try:
            # Verifica se os elementos estão presentes na página
            elemento_pai = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'items')]"))
            )

            # Executa o código para pegar os elementos filhos
            elementos_filhos = elemento_pai.find_elements(By.XPATH, "./*")
            for elemento in elementos_filhos:
                soup = BeautifulSoup(elemento.get_attribute('innerHTML'), 'html.parser')
                div = soup.find('div')
                if div:
                    cor = div.get('class')[1] if len(div.get('class')) > 1 else None
                    aria_label = div.get('aria-label')
                    numero = div.text
                    if aria_label:
                        data, hora = aria_label.split(", ")
                    else:
                        data, hora = None, None
                    print(f"Cor: {cor}, Data: {data}, Hora: {hora}, Número: {numero}")

            # Armazena o código HTML atual da página
            html_atual = driver.page_source

            # Verifica se houve mudanças na página
            if html_atual != html_antigo:
                html_antigo = html_atual
                driver.refresh()

            # Espera por 30 segundos antes de verificar novamente
            time.sleep(20)

        except Exception as e:
            print(e)
            driver.quit()
            break

except KeyboardInterrupt:
    print("Programa interrompido pelo usuário")
    driver.quit()


Cor: red, Data: 21/08/2023, Hora: 13:33:37, Número: 4
Cor: red, Data: 21/08/2023, Hora: 13:34:03, Número: 2
Cor: green, Data: 21/08/2023, Hora: 13:34:30, Número: 13
Cor: green, Data: 21/08/2023, Hora: 13:34:56, Número: 14
Cor: red, Data: 21/08/2023, Hora: 13:35:23, Número: 6
Cor: green, Data: 21/08/2023, Hora: 13:35:49, Número: 13
Cor: green, Data: 21/08/2023, Hora: 13:36:16, Número: 13
Cor: green, Data: 21/08/2023, Hora: 13:36:42, Número: 14
Cor: red, Data: 21/08/2023, Hora: 13:37:09, Número: 1
Cor: red, Data: 21/08/2023, Hora: 13:37:36, Número: 7
Cor: green, Data: 21/08/2023, Hora: 13:38:02, Número: 12
Cor: red, Data: 21/08/2023, Hora: 13:38:29, Número: 2
Cor: red, Data: 21/08/2023, Hora: 13:38:55, Número: 4
Cor: green, Data: 21/08/2023, Hora: 13:39:22, Número: 13
Cor: red, Data: 21/08/2023, Hora: 13:39:48, Número: 7
Cor: red, Data: 21/08/2023, Hora: 13:40:15, Número: 5
Cor: red, Data: 21/08/2023, Hora: 13:40:41, Número: 7
Cor: green, Data: 21/08/2023, Hora: 13:41:08, Número: 8
Cor: 

In [ ]:
import os
import time
import tkinter as tk
from bs4 import BeautifulSoup
from selenium.webdriver import Edge
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from collections import deque
from strat import Strat
from face import FaceGUI

# Cria uma instância da classe Strat
strategia = Strat()

# Cria a janela principal do Tkinter
root = tk.Tk()
root.title("Robô de Apostas")

# Instancia a classe FaceGUI, passando a janela e a instância da classe Strat
face_gui = FaceGUI(root, strategia)

# Iniciar a atualização da interface
face_gui.atualizar()

# Configura as opções do Edge
options = Options()
options.add_argument("--headless")

# Configura o serviço do EdgeDriver
service = Service(EdgeChromiumDriverManager().install())

# Cria uma nova instância do Microsoft Edge
driver = Edge(service=service, options=options)

driver.implicitly_wait(10)

url = "https://www.arbety.com/games/double"
driver.get(url)

# Armazena o código HTML atual da página
html_antigo = driver.page_source

# Definir o tamanho máximo da fila
TAMANHO_MAXIMO = 20

# Criar uma fila com tamanho máximo
fila_dados_impressos = deque(maxlen=TAMANHO_MAXIMO)

try:
    while True:
        try:
            # Verifica se os elementos estão presentes na página
            elemento_pai = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'items')]"))
            )

            # Executa o código para pegar os elementos filhos
            elementos_filhos = elemento_pai.find_elements(By.XPATH, "./*")

            # Lista para armazenar os dados atuais na página
            dados_atuais = []

            for elemento in elementos_filhos:
                try:
                    soup = BeautifulSoup(elemento.get_attribute("innerHTML"), "html.parser")
                except StaleElementReferenceException:
                    print("Elemento obsoleto encontrado. Continuando...")
                    continue

                div = soup.find("div")
                if div:
                    cor = div.get("class")[1] if len(div.get("class")) > 1 else None
                    aria_label = div.get("aria-label")
                    numero = div.text
                    if aria_label:
                        data, hora = aria_label.split(", ")
                    else:
                        data, hora = None, None
                    linha_dados = f"Cor: {cor}, Data: {data}, Hora: {hora}, Número: {numero}"
                    dados_atuais.append(linha_dados)

            # Adiciona os novos dados à fila
            fila_dados_impressos.extend(dados_atuais[-TAMANHO_MAXIMO:])

            # Chama o método processar_cores para processar as cores e tomar decisões de apostas
            strategia.processar_cores(fila_dados_impressos)

            # Atualiza a interface gráfica com os novos dados
            face_gui.atualizar_interface()

            # Armazena o código HTML atual da página
            html_atual = driver.page_source

            # Verifica se houve mudanças na página
            if html_atual != html_antigo:
                html_antigo = html_atual
                driver.refresh()

            # Espera por 15 segundos antes de verificar novamente
            time.sleep(15)

        except NoSuchElementException as e:
            print(e)
            driver.quit()
            break

except KeyboardInterrupt:
    print("Programa interrompido pelo usuário")
    driver.quit()

# Iniciar o loop principal do Tkinter
root.mainloop()


In [6]:
from collections import deque

class Strat:
    def __init__(self):
        self.lista_cores = deque(maxlen=10)
        self.valor_aposta = 0
        self.pode_apostar = False
        self.ultima_lista_cores = []
        self.analise_pendente = None
        self.saldo = 50
        self.acertos = 0
        self.perdas = 0
        self.robo_ligado = False

    def definir_acerto(self, cores):
        padroes = {
            ('red', 'green', 'red'): 'green',
            ('green', 'red', 'green'): 'red',
            ('green', 'red', 'red', 'green'): 'red',
            ('red', 'green', 'green', 'red'): 'green',
            ('red', 'red'): 'red',
            ('green', 'green',): 'green',
            ('green', 'white', 'green'): 'green',
        }
        return padroes.get(cores, None)

    def fazer_aposta(self, cor):
        if self.robo_ligado and self.saldo >= self.valor_aposta:
            self.saldo -= self.valor_aposta
            return cor, self.valor_aposta
        else:
            return None, None

    def registrar_ganho(self, cor_vencedora):
        ganho = self.valor_aposta
        if cor_vencedora == 'red' or cor_vencedora == 'green':
            ganho *= 2
        elif cor_vencedora == 'white':
            ganho *= 14
        self.saldo += ganho
        self.acertos += 1

    def registrar_perda(self):
        self.saldo -= self.valor_aposta
        self.perdas -= 1

    def realizar_aposta(self, cor_aposta, resultado):
        ganhou = resultado == cor_aposta
        if ganhou:
            self.registrar_ganho(resultado)
        else:
            self.registrar_perda()

    def verificar_meta(self):
        meta = 150
        if self.saldo >= meta:
            print("Meta alcançada!")

    def analisar_cores(self):
        if not self.pode_apostar:
            print("Aguardando mais cores...")
            return
        
        for i in range(4, 1, -1):
            cores = tuple(list(self.lista_cores)[-i:])
            cor_aposta = self.definir_acerto(cores)
        
            if cor_aposta:
                print("Encontrado um Padrão..", f"Padrão encontrado: {cores} -> {cor_aposta}")
                self.analise_pendente = cor_aposta
                break
        else:
            print("Analisando...")

    def definir_valor_aposta(self, valor):
        self.valor_aposta = valor

    def alternar_status_robo(self):
        self.robo_ligado = not self.robo_ligado

    def adicionar_cor(self, cor):
        self.lista_cores.append(cor)

    def processar_cores(self, fila_dados_impressos):
        for item in fila_dados_impressos:
            cor = item.split(",")[0].split(":")[1].strip()
            self.lista_cores.append(cor)

        if len(self.lista_cores) > 10:
            self.lista_cores = deque(list(self.lista_cores)[-10:], maxlen=10)

        if list(self.lista_cores) != self.ultima_lista_cores:
            print("Cores atuais:", list(self.lista_cores))
            self.ultima_lista_cores = list(self.lista_cores)
            self.pode_apostar = True

            if self.analise_pendente:
                resultado_real = self.lista_cores[-1]
                print("Vamos iniciar as apostas..")
                cor_apostada, valor_aposta = self.fazer_aposta(self.analise_pendente)
                self.realizar_aposta(cor_apostada, resultado_real)
                self.analise_pendente = None

            self.analisar_cores()
        else:
            print("Aguardando novas cores...")
            self.pode_apostar = False
            

In [ ]:
import tkinter as tk
from strat import Strat  # Certifique-se de importar a classe Strat

class FaceGUI:
    def __init__(self, master, strat_instance):
        self.master = master
        self.strat = strat_instance
        master.title("Robô Arbety")
        master.geometry("800x600")
        master.configure(bg="#163340")

        # Title label
        self.title_label = tk.Label(master, text="DOUBLE ARBETY", font=("Arial", 24), bg="#163340", fg="white")
        self.title_label.place(relx=0.5, rely=0.1, anchor=tk.CENTER)

        # Bet value entry
        self.aposta_label = tk.Label(master, text="Valor da Aposta:", font=("Arial", 12), bg="#163340", fg="white")
        self.aposta_label.place(relx=0.3, rely=0.2, anchor=tk.CENTER)
        self.aposta_entry = tk.Entry(master)
        self.aposta_entry.place(relx=0.5, rely=0.2, anchor=tk.CENTER)

        # Robot status button
        self.robo_status_button = tk.Button(master, text="Robo Ligado", command=self.toggle_robo_status)
        self.robo_status_button.place(relx=0.5, rely=0.3, anchor=tk.CENTER)

        # Bankroll label
        self.banca_label = tk.Label(master, text="Banca: " + str(self.strat.saldo), font=("Arial", 12), bg="#163340", fg="white")
        self.banca_label.place(relx=0.5, rely=0.4, anchor=tk.CENTER)

        # Profit and loss label
        self.lucro_label = tk.Label(master, text="Lucro: 0 | Perda: 0", font=("Arial", 12), bg="#163340", fg="white")
        self.lucro_label.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

        # Correct and incorrect guesses label
        self.acertos_label = tk.Label(master, text="Acertos: 0 | Erros: 0", font=("Arial", 12), bg="#163340", fg="white")
        self.acertos_label.place(relx=0.5, rely=0.6, anchor=tk.CENTER)

        # Canvas for balls
        self.canvas_bolas = tk.Canvas(master, width=520, height=60, bg="#163340")
        self.canvas_bolas.place(relx=0.5, rely=0.8, anchor=tk.CENTER)

        # Periodically update the GUI with the latest information
        self.update_gui()

    def toggle_robo_status(self):
        # Toggling the robot status and updating the betting value
        self.strat.alternar_status_robo()
        if self.strat.robo_ligado:
            self.robo_status_button.config(text="Robo Ligado")
        else:
            self.robo_status_button.config(text="Robo Desligado")
        aposta_valor = float(self.aposta_entry.get())
        self.strat.definir_valor_aposta(aposta_valor)

    def desenhar_bolas(self):
        self.canvas_bolas.delete("all")
        x = 10
        for cor in self.strat.lista_cores:
            self.canvas_bolas.create_oval(x, 10, x + 10, 20, fill=cor)
            x += 15

    def update_gui(self):
        # Updating the GUI labels with the latest information
        self.banca_label.config(text="Banca: " + str(self.strat.saldo))
        lucro = self.strat.acertos * self.strat.valor_aposta - abs(self.strat.perdas) * self.strat.valor_aposta
        self.lucro_label.config(text="Lucro: " + str(lucro) + " | Perda: " + str(abs(self.strat.perdas)))
        self.acertos_label.config(text="Acertos: " + str(self.strat.acertos) + " | Erros: " + str(abs(self.strat.perdas)))
        self.desenhar_bolas()
        self.master.after(1000, self.update_gui)  # Update every 1 second

# Exemplo de uso (você pode executar este código localmente para testar a GUI)
strat_instance = Strat()
root = tk.Tk()
face_gui = FaceGUI(root, strat_instance)
root.mainloop()
